## 세 개 모델의 예측 값 조합
- 확률 값의 평균
- 최대 투표 범주 확인 (보팅)

In [3]:
import numpy as np
from scipy

# 예측 확률 값의 평균을 구하는 함수
def mean_predictions(probas):
    """
    예측 확률 값의 평균을 구하는 함수
    : param probas: 예측 확률 값. 2차원 행렬.
    : return: 평균 확률
    """
    return np.mean(probas)


# 예측 확률 값의 최대 투표 범주를 구하는 함수
def max_voting(preds):
    """
    예측 확률 값의 최대 투표 범주를 구하는 함수
    : param probas: 예측 확률 값. 2차원 행렬
    : return: 최대 투표 범주
    """
    idxs = np.aramax(preds, axis = 1)
    return np.take_along_axis(preds, idxs[:, None], axis = 1)


# 예측 확률 값의 순위를 사용
def rank_mean(probas):
    """
    예측 확률 값의  순위 평균을 구하는 함수
    : param probas: 예측 확률 값. 2차원 행렬
    : return: 순위 평균
    """
    ranked = []
    for i in range(probas.shape[1]):
        rank_data = stats.rankdata(probas[:,i])
        ranked.append(rank_data)
        
    ranked = np.column_stack(ranked)
    return np.mean(ranked, axis = 1)

## rankdata 만지기

In [15]:
from scipy.stats import rankdata
import scipy

a = np.array([ 5, 0.3,  0.4, 1, 4, 2, 3, 42])

# empty_like() : 주어진 데이터와 같은 형태를 갖는 것을 만듦
almost_ranks = np.empty_like(a)

# argsort() : 정렬된 행렬의 인덱스를 반환
almost_ranks[np.argsort(a)] = np.arange(len(a))
print(almost_ranks)
print(scipy.stats.rankdata(a))

[6. 0. 1. 2. 5. 3. 4. 7.]
[7. 1. 2. 3. 6. 4. 5. 8.]


In [16]:
import numpy as np
from functools import partial
from scipy.optimize import fmin
from sklearn import metrics

class OptimizeAUC:
    """
    AUC를 최적화하는 클래스
    
    약간의 변경으로 임의의 모델, 임의의 메트릭, 임의의 예측 값의 타입에 대한
    최적의 가중치를 찾는 클래스로 만들 수 있다.
    """
    def __init__(self):
        self.coef_ = 0
        
    def _auc(self, coef, X, y):
        """
        AUC를 계산하고 반환하는 함수
        : param coef: 가중치 목록, 모델의 수와 같은 길이
        : param x: 예측 값, 2차원 행렬
        : param y: 타겟 값, 1차원 벡터
        """
        # 가중치를 각 예측 값의 열에 곱한다.
        # 예를 들어, 첫 번째 가중치는 예측 값 행렬의 첫 번째 열에 곱하고,
        # 두 번째 가중치는 예측 값 행렬의 두 번째 열에 곱한다.
        x_coef = X*coef
        
        # 각 행의 합으로 예측 값을 생성한다.
        predictions = np.sum(x_coef, axis = 1)
        
        # auc 점수를 계산한다.
        auc_score = metrics.roc_auc_score(y, predictions)
        
        # 음의 auc 점수를 반환한다.
        return -1.0 * auc_score
        
    def fit(self, X, y):
        # 하이퍼파라미터 최적화 장에서 다뤘던 partial 함수이다.
        loss_partial = partial(self._auc, X=X, y=y)

        # dirichlet 분포로 가중치 초기화.
        # 가중치의 합이 1 인 임의의 분포를 사용할 수 있다.
        initial_coef = np.random.dirichlet(np.ones(X.shape[1]),
                                            size=1)
        # 손실 함수 (auc) 최적화를 위해 scipy 의 fmin 함수를 사용한다.
        self.coef_ = fmin(loss_partial, initial_coef, disp=True)

    def predict(self, X):
        # _auc 함수와 비슷하다.
        x_coef = X * self.coef_
        predictions = np.sum(x_coef, axis=1)
        return predictions

In [31]:
import xgboost as xgb
from sklearn.datasets import make_classification
from sklearn import ensemble
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection

# 10k 샘플과 25 개의 피쳐로 이진분류 학습 데이터를 생성한다.
X, y = make_classification(n_samples=10000, n_features=25)

# 데이터를 두 개의 폴드로 나눈다.
xfold1, xfold2, yfold1, yfold2 = model_selection.train_test_split(
 X,
 y,
 test_size=0.5,
 stratify=y
)

# 폴드 1 데이터로 학습 후, 폴드 2 데이터에 대한 예측을 한다.
# 로지스틱 회귀, 랜덤포레스트, xgboost 의 3 개의 모델을 학습한다.
logreg = linear_model.LogisticRegression()
rf = ensemble.RandomForestClassifier()
xgbc = xgb.XGBClassifier()

# 모든 모델을 폴드 1 데이터로 학습한다.
logreg.fit(xfold1, yfold1)
rf.fit(xfold1, yfold1)
xgbc.fit(xfold1, yfold1)

# 모든 모델을 폴드 2 데이터로 예측한다.
# 범주 1 에 대한 예측 값(확률)들을 추출한다.
pred_logreg = logreg.predict_proba(xfold2)[:, 1]
pred_rf = rf.predict_proba(xfold2)[:, 1]
pred_xgbc = xgbc.predict_proba(xfold2)[:, 1]


## 평균 예측 값 계산
# 가장 단순한 앙상블로 평균 예측 값을 계산한다.
avg_pred = (pred_logreg + pred_rf + pred_xgbc)/3


## 모든 예측 값의 2 차원 행렬
# 4번째 열이 앙상블 평균 예측 값
fold2_preds = np.column_stack((
 pred_logreg,
 pred_rf,
 pred_xgbc,
 avg_pred
))


# 개별 AUC 값을 계산하여 저장한다.
aucs_fold2 = []
for i in range(fold2_preds.shape[1]):
    auc = metrics.roc_auc_score(yfold2, fold2_preds[:, i])
    aucs_fold2.append(auc)
    
print(f"Fold-2: LR AUC = {aucs_fold2[0]}")
print(f"Fold-2: RF AUC = {aucs_fold2[1]}")
print(f"Fold-2: XGB AUC = {aucs_fold2[2]}")
print(f"Fold-2: Average Pred AUC = {aucs_fold2[3]}")
# 폴드를 바꾸어서 동일하게 학습/예측을 수행한다.
# 아래와 같이 코드를 반복하는 것은 이상적이지 않다.
# 반복되는 코드가 있다면 함수를 만들어 수행
# 폴드 2 데이터로 학습 후, 폴드 1 데이터에 대한 예측을 한다.

## 폴드 2 학습
logreg = linear_model.LogisticRegression()
rf = ensemble.RandomForestClassifier()
xgbc = xgb.XGBClassifier()

logreg.fit(xfold2, yfold2)
rf.fit(xfold2, yfold2)
xgbc.fit(xfold2, yfold2)

pred_logreg = logreg.predict_proba(xfold1)[:, 1]
pred_rf = rf.predict_proba(xfold1)[:, 1]
pred_xgbc = xgbc.predict_proba(xfold1)[:, 1]
avg_pred = (pred_logreg + pred_rf + pred_xgbc)/3

fold1_preds = np.column_stack((
 pred_logreg,
 pred_rf,
 pred_xgbc,
 avg_pred
))

aucs_fold1 = []
for i in range(fold1_preds.shape[1]):
    auc = metrics.roc_auc_score(yfold1, fold1_preds[:, i])
    aucs_fold1.append(auc)
    
print(f"Fold-1: LR AUC = {aucs_fold1[0]}")
print(f"Fold-1: RF AUC = {aucs_fold1[1]}")
print(f"Fold-1: XGB AUC = {aucs_fold1[2]}")
print(f"Fold-1: Average prediction AUC = {aucs_fold1[3]}")


# (위에서 만든)최적화 클래스로 최적 가중치를 찾는다.
opt = OptimizeAUC()

# 앞서 추가한 평균 예측 값을 제외하고 최적화 함수를 학습한다.
opt.fit(fold1_preds[:, :-1], yfold1)
opt_preds_fold2 = opt.predict(fold2_preds[:, :-1])
auc = metrics.roc_auc_score(yfold2, opt_preds_fold2)
print(f"Optimized AUC, Fold 2 = {auc}")
print(f"Coefficients = {opt.coef_}")


opt = OptimizeAUC()
opt.fit(fold2_preds[:, :-1], yfold2)
opt_preds_fold1 = opt.predict(fold1_preds[:, :-1])
auc = metrics.roc_auc_score(yfold1, opt_preds_fold1)
print(f"Optimized AUC, Fold 1 = {auc}")
print(f"Coefficients = {opt.coef_}")

C:\Users\Roh Seung Chan\anaconda3\envs\algo\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:53:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold-2: LR AUC = 0.968479279306955
Fold-2: RF AUC = 0.9881588496866552
Fold-2: XGB AUC = 0.9889998918397231
Fold-2: Average Pred AUC = 0.987911489053412


C:\Users\Roh Seung Chan\anaconda3\envs\algo\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:53:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fold-1: LR AUC = 0.9667666349225853
Fold-1: RF AUC = 0.9868090862312608
Fold-1: XGB AUC = 0.9874648479100105
Fold-1: Average prediction AUC = 0.9862474847935611
Optimization terminated successfully.
         Current function value: -0.987686
         Iterations: 62
         Function evaluations: 130
Optimized AUC, Fold 2 = 0.9895599732735316
Coefficients = [-0.00144709  0.02163006  0.17085201]
Optimization terminated successfully.
         Current function value: -0.989600
         Iterations: 89
         Function evaluations: 166
Optimized AUC, Fold 1 = 0.9876484483800279
Coefficients = [-0.00346526  0.06029454  0.89383637]
